In [2]:
%load_ext dotenv
%dotenv

In [3]:
import openai
import os
import pandas as pd
import stanza
import time
from datasets import load_dataset
from tqdm import tqdm

In [4]:
# Only needs to be run one time
# stanza.download('en')

In [74]:
split_name = 'train'
en_csqa = load_dataset('commonsense_qa', split=split_name)

Found cached dataset commonsense_qa (/mnt/nas2/kikiputri/cache/commonsense_qa/default/1.0.0/28d68f56649a7f0c23bc68eae850af914aa03f95f810011ae8cf58cc5ff5051b)


### Concept Relevancy Classifier

#### Post-process Prediction

In [88]:
out_parent_dir = "/mnt/nas2/kikiputri/id-csqa/dataset/relevancy/"
relevancy_df = pd.read_csv(out_parent_dir + split_name + "_su_step1.csv", index_col=0)

In [89]:
relevancy_df

,q_concept,option_a,option_b,option_c,option_d,option_e,names,answer
q_id,,,,,,,,
075e483d21c29a511267ef62bedc0461,yes,NaN,NaN,NaN,NaN,NaN,[],A
61fe6e879ff18686d7552425a36344c8,yes,NaN,NaN,NaN,NaN,NaN,"[('Sammy', 'no')]",B
4c1cb0e95b99f72d55c068ba0255c54d,yes,NaN,NaN,NaN,NaN,NaN,[],A
02e821a3e53cb320790950aab4489e85,yes,NaN,no,NaN,NaN,NaN,[],D
23505889b94e880c3e89cff4ba119860,yes,NaN,NaN,NaN,NaN,NaN,[],C
...,...,...,...,...,...,...,...,...
f1b2a30a1facff543e055231c5f90dd0,no,NaN,NaN,NaN,NaN,NaN,[],E
a63b4d0c0b34d6e5f5ce7b2c2c08b825,yes,NaN,NaN,NaN,NaN,NaN,[],D
22d0eea15e10be56024fd00bb0e4f72f,yes,NaN,NaN,NaN,NaN,NaN,[],A


In [90]:
general_questions = []
for item in tqdm(en_csqa):
    q_id = item['id']

    # Check name relevancy
    is_name_irrelevant = True if "'no'" in relevancy_df.loc[q_id, 'names'] else False

    # Check options relevancy
    options = [
        relevancy_df.loc[q_id, 'option_a'], relevancy_df.loc[q_id, 'option_b'],
        relevancy_df.loc[q_id, 'option_c'], relevancy_df.loc[q_id, 'option_d'],
        relevancy_df.loc[q_id, 'option_e']
    ]
    is_option_irrelevant = True if 'no' in options else False

    # Check q concept relevancy
    is_concept_relevant = True if relevancy_df.loc[q_id, 'q_concept'] == 'yes' else False

    if not is_name_irrelevant and not is_option_irrelevant and is_concept_relevant:
        general_questions.append(item)

100%|██████████| 9741/9741 [00:01<00:00, 6845.85it/s]


In [91]:
len(general_questions)

5351

In [95]:
general_q_df = pd.DataFrame(general_questions)

In [96]:
general_q_df

,id,question,question_concept,choices,answerKey
0,075e483d21c29a511267ef62bedc0461,The sanctions against the school were a punish...,punishing,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
1,4c1cb0e95b99f72d55c068ba0255c54d,To locate a choker not located in a jewelry bo...,choker,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
2,23505889b94e880c3e89cff4ba119860,"The fox walked from the city into the forest, ...",fox,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",C
3,e8a8b3a2061aa0e6d7c6b522e9612824,What home entertainment equipment requires cable?,cable,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",D
4,09555c056f3cf0b7e0b84d8df4be1db7,The only baggage the woman checked was a draws...,drawstring bag,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",E
...,...,...,...,...,...
5346,47011436a95298809c57ce5f59ebf1d1,What type of homes are likely to have an attic?,attic,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",C
5347,f1eb6055fa8a1ec94fa6d710d9a6741b,Something that you need to have inside of you ...,opening business,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",C
5348,a63b4d0c0b34d6e5f5ce7b2c2c08b825,Where might you find a chair at an office?,chair,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",D
5349,22d0eea15e10be56024fd00bb0e4f72f,Where would you buy jeans in a place with a la...,jeans,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A


In [97]:
out_parent_dir = "/mnt/nas2/kikiputri/id-csqa/dataset/relevancy/data_result/"
relevancy_df.to_csv(out_parent_dir + split_name + "_general_su.csv", index=False)

#### Model Predict

In [98]:
ner_pipeline = stanza.Pipeline('en', processors='tokenize,ner', device=7)

2023-07-07 15:47:54 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-07-07 15:47:55 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2023-07-07 15:47:55 INFO: Using device: 7
2023-07-07 15:47:55 INFO: Loading: tokenize
2023-07-07 15:47:57 INFO: Loading: ner
2023-07-07 15:47:59 INFO: Done loading processors!


In [7]:
def extract_names(sentence):
    ner_results = ner_pipeline(sentence)
    return [ent.text for ent in ner_results.ents if ent.type == 'PERSON']


def extract_locations(sentence):
    ner_results = ner_pipeline(sentence)
    return [ent.text for ent in ner_results.ents if ent.type in ['GPE', 'LOC']]

In [8]:
openai.api_key = os.environ['OPENAI_API_KEY']
openai.organization = os.environ['OPENAI_UILAB_KEY']
# response_history = {}

In [9]:
def get_input_prompt(concept, concept_type, location_name):
    end_prompt = "Answer with only 'yes' or 'no'."
    
    if concept_type == 'location':
        return f"Is {concept} located in {location_name}? {end_prompt}"
    
    if concept_type == 'name':
        return f"Is the name \"{concept}\" common in {location_name}? {end_prompt}"

    return f"Does {concept} commonly found in {location_name}? {end_prompt}"


def get_openai_chat_completion(input_prompt, model_name, temp=0.2):
    return openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {
                'role': 'user',
                'content': input_prompt
            }
        ],
        temperature=temp
    )


def get_openai_relevancy(input_prompt, model_name):
    if input_prompt in response_history:
        return response_history[input_prompt]
    
    try:
        completion = get_openai_chat_completion(input_prompt, model_name)
    except openai.error.RateLimitError:
        # print("OpenAI RateLimitError, wait for 1 minute...")
        time.sleep(60)
        completion = get_openai_chat_completion(input_prompt, model_name)
    except openai.error.ServiceUnavailableError:
        # print("OpenAI ServiceUnavailableError, wait for 1 minute...")
        time.sleep(60)
        completion = get_openai_chat_completion(input_prompt, model_name)

    response = completion.choices[0].message.content.strip().lower()
    
    if response in ["yes.", "no."]:
        response = response.replace(".", "")
    
    response_history[input_prompt] = response

    return response

In [30]:
en_csqa[0]

{'id': '075e483d21c29a511267ef62bedc0461',
 'question': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?',
 'question_concept': 'punishing',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']},
 'answerKey': 'A'}

In [35]:
model_name = "gpt-3.5-turbo"
relevancy_data = {
    'q_id': [], 'q_concept': [],
    'option_a': [], 'option_b': [], 'option_c': [], 'option_d': [], 'option_e': [],
    'names': [], 'answer': []
}
option_idxs = ['option_a', 'option_b', 'option_c', 'option_d', 'option_e']
for item in tqdm(en_csqa):
    relevancy_data['q_id'].append(item['id'])

    input_prompt = get_input_prompt(item['question_concept'], "other", "West Java")
    rel = get_openai_relevancy(input_prompt, model_name)
    relevancy_data['q_concept'].append(rel)

    for option_idx, choice in zip(option_idxs, item['choices']['text']):
        locations = extract_locations(choice)
        if len(locations) > 0:
            input_prompt = get_input_prompt(choice, "location", "West Java")
            rel = get_openai_relevancy(input_prompt, model_name)
            relevancy_data[option_idx].append(rel)
        else:
            relevancy_data[option_idx].append(None)
        
    names = extract_names(item['question'])
    names_rel = []
    for name in names:
        input_prompt = get_input_prompt(name, "name", "West Java")
        rel = get_openai_relevancy(input_prompt, model_name)
        names_rel.append((name, rel))
    relevancy_data['names'].append(names_rel)

    relevancy_data['answer'].append(item['answerKey'])

100%|██████████| 9741/9741 [10:14<00:00, 15.86it/s]  


In [36]:
relevancy_df = pd.DataFrame(relevancy_data)

In [37]:
relevancy_df

,q_id,q_concept,option_a,option_b,option_c,option_d,option_e,names,answer
0,075e483d21c29a511267ef62bedc0461,yes,None,None,None,None,None,[],A
1,61fe6e879ff18686d7552425a36344c8,yes,None,None,None,None,None,"[(Sammy, no)]",B
2,4c1cb0e95b99f72d55c068ba0255c54d,yes,None,None,None,None,None,[],A
3,02e821a3e53cb320790950aab4489e85,yes,None,no,None,None,None,[],D
4,23505889b94e880c3e89cff4ba119860,yes,None,None,None,None,None,[],C
...,...,...,...,...,...,...,...,...,...
9736,f1b2a30a1facff543e055231c5f90dd0,no,None,None,None,None,None,[],E
9737,a63b4d0c0b34d6e5f5ce7b2c2c08b825,yes,None,None,None,None,None,[],D
9738,22d0eea15e10be56024fd00bb0e4f72f,yes,None,None,None,None,None,[],A
9739,7c55160a4630de9690eb328b57a18dc2,yes,None,None,None,None,None,"[(John, no)]",A


In [38]:
out_parent_dir = "/mnt/nas2/kikiputri/id-csqa/dataset/relevancy/"
relevancy_df.to_csv(out_parent_dir + split_name + "_su_step1.csv", index=False)

In [39]:
resp_history_df = pd.DataFrame({'prompt': response_history.keys(), 'response': response_history.values()})

In [40]:
resp_history_df

,prompt,response
0,Does revolving door commonly found in West Jav...,no
1,Does people commonly found in West Java? Answe...,yes
2,Does magazines commonly found in West Java? An...,no
3,Does hamburger commonly found in West Java? An...,yes
4,Does farmland commonly found in West Java? Ans...,yes
...,...,...
2533,"Is the name ""Lud"" common in West Java? Answer ...",no
2534,Does dining room table commonly found in West ...,no
2535,Does restroom commonly found in West Java? Ans...,yes
2536,"Is the name ""Mike"" common in West Java? Answer...",no


In [41]:
resp_history_df.to_csv(out_parent_dir + "gpt-3.5-history-su-230704.csv", index=False)